If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets as well as other dependencies. Uncomment the following cell and run it.

In [ ]:
! pip install datasets transformers rouge-score nltk
import nltk
nltk.download('punkt')
from datasets import load_metric

metric = load_metric("rouge")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 6.7 MB/s 
     |████████████████████████████████| 4.4 MB 61.9 MB/s 
     |████████████████████████████████| 212 kB 89.5 MB/s 
     |████████████████████████████████| 101 kB 12.9 MB/s 
     |████████████████████████████████| 140 kB 96.1 MB/s 
     |████████████████████████████████| 1.1 MB 80.7 MB/s 
     |████████████████████████████████| 596 kB 84.3 MB/s 
     |████████████████████████████████| 127 kB 78.6 MB/s 
     |████████████████████████████████| 6.6 MB 68.0 MB/s 
     |████████████████████████████████| 144 kB 79.5 MB/s 
     |████████████████████████████████| 94 kB 4.6 MB/s 
     |████████████████████████████████| 271 kB 98.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Fine-tuning a model on a summarization task

In [ ]:
model_checkpoint = "t5-small"

## Loading the dataset

In [ ]:
mydataset = {'document':['Which animal is known as the \'Ship of the Desert\"?', 'How many days are there in a week?', 'How many hours are there in a day?'], 'summary':['Camel', '7 days', '24 hours'], 'id':[1,2,3]}

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
import pickle
with open("/content/gdrive/MyDrive/clarifying/bart_inputs.pkl", "rb") as f_bart_inputs:
  train_examples, val_examples, test_examples = pickle.load(f_bart_inputs)

In [ ]:
import pickle
modes = ["only query", "only actual", "pair", "choices"]
mode = modes[2]
inputs_prefix = "/content/gdrive/MyDrive/clarifying/turk_bart_inputs_"
with open(inputs_prefix + 'train' + '_'  +mode+'.pkl', "rb") as f_bart_inputs:
  train_examples = pickle.load(f_bart_inputs)
with open(inputs_prefix + 'val' + '_'  +mode+'.pkl', "rb") as f_bart_inputs:
  val_examples = pickle.load(f_bart_inputs)
with open(inputs_prefix + 'test' + '_'  +mode+'.pkl', "rb") as f_bart_inputs:
  test_examples = pickle.load(f_bart_inputs)

In [ ]:
for k,v in train_examples.items():
  print(k,v[0])

src_e_list ['urinari', 'infect']
tgt_e_set {'caus', 'complic'}
tgt_fq What puts me at risk for recurrent urinary tract infections?
query urinary infections
clarification Do you want to know about causes and complications of urinary infections?
id_cands 1056 1057
cands ['My healthcare practitioner said I had symptoms of a urinary tract infection and prescribed antibiotics without performing a urine culture. Why?', 'What puts me at risk for recurrent urinary tract infections?']
dialogue user: urinary infections;


tgt_question Do you want to know about causes and complications of urinary infections?


## Preprocessing the data

In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "my task: "
else:
    prefix = ""

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task

In [ ]:
import torch
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, contents):
        self.contents = contents

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.contents.items()}
        return item

    def __len__(self):
        return len(self.contents[ list(self.contents.keys())[0] ])
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["tgt_question"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocessed_mydataset = preprocess_function(train_examples)
my_dataset_train = MyDataset(preprocessed_mydataset)
preprocessed_mydataset = preprocess_function(val_examples)
my_dataset_val = MyDataset(preprocessed_mydataset)
preprocessed_mydataset = preprocess_function(test_examples)
my_dataset_test = MyDataset(preprocessed_mydataset)

## Fine-tuning the model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penal

In [ ]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=50,
    predict_with_generate=True,
    fp16=True
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import torch

In [ ]:
print( tokenizer.decode( my_dataset_test[0]['input_ids'] ) )
print( tokenizer.decode( my_dataset_test[0]["labels"] ) )

my task: user: type genes; what the user actually means: Is my blood type (ABO) related to my HLA genes and antigens?;</s>
Do you want to know about HLA genes and antigens in relation to blood type ABO?</s>


In [ ]:
model.to("cpu")
sequences = []
for i in range(len(my_dataset_test)):
  #input_ids = tokenizer("How many minutes are there in a year?", return_tensors="pt").input_ids
  #input_ids.to("cuda:0")
  input_text = tokenizer.decode( my_dataset_test[i]['input_ids'] )
  sequence_ids = model.generate( tokenizer(input_text, return_tensors="pt").input_ids )
  sequence = tokenizer.batch_decode(sequence_ids)
  sequences.append(sequence)
  print(input_text)
  print(sequence)  
  #input()

my task: user: type genes; what the user actually means: Is my blood type (ABO) related to my HLA genes and antigens?;</s>
['<pad> Do you want to know about type genes and antigens?</s>']
my task: user: cystic fibrosis; what the user actually means: Does this test detect all genetic causes of cystic fibrosis?;</s>
['<pad> Do you want to know about cystic fibrosis?</s>']
my task: user: worms stool; what the user actually means: Are those parasitic worms I'm seeing in my stool sample?;</s>
['<pad> Do you want to know about parasitic worms in stool sample?</s>']
my task: user: gene repair; what the user actually means: How do you know a mismatch repair gene variant is harmful (pathogenic)?;</s>
['<pad> Do you want to know about a mismatch repair gene variant</s>']
my task: user: allergy symptoms; what the user actually means: My allergy symptoms are generally mild. How serious is this really?;</s>
['<pad> Do you want to know about allergy symptoms?</s>']
my task: user: thyroglobulin level

In [ ]:
test_examples['pred_q'] = sequences
outputs_prefix = "/content/gdrive/MyDrive/clarifying/turk_bart_outputs_"
with open(outputs_prefix +mode+'.pkl', "wb") as f_bart_output:
  pickle.dump(test_examples, f_bart_output)

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
my_tokenized_dataset = preprocess_function(my_dataset[:1])

In [ ]:

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=my_dataset_train,
    eval_dataset=my_dataset_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using cuda_amp half precision backend


We can now finetune our model by just calling the `train` method:

In [ ]:
model.to('cuda:0')

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 451
  Num Epochs = 50
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1450


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.046565,11.390100,2.735400,9.514000,9.428300,18.207800
2,No log,1.637487,54.726000,37.637400,52.391600,52.240100,14.928600
3,No log,1.430913,74.330500,57.989100,72.314300,72.251100,14.045500
4,No log,1.338426,74.940200,58.476900,72.735000,72.727300,14.824700
5,No log,1.290381,75.002000,58.286500,72.585700,72.583700,15.013000
6,No log,1.262884,75.783900,59.239000,73.549800,73.506900,14.759700
7,No log,1.241023,75.468000,59.086900,73.483500,73.376500,15.136400
8,No log,1.226495,76.227200,59.997900,74.060000,74.009700,14.766200
9,No log,1.213506,75.931100,59.785700,73.797000,73.771700,14.759700
10,No log,1.205111,76.040600,59.769800,73.738700,73.683300,15.077900


***** Running Evaluation *****
  Num examples = 154
  Batch size = 16
***** Running Evaluation *****
  Num examples = 154
  Batch size = 16
***** Running Evaluation *****
  Num examples = 154
  Batch size = 16
***** Running Evaluation *****
  Num examples = 154
  Batch size = 16
***** Running Evaluation *****
  Num examples = 154
  Batch size = 16
***** Running Evaluation *****
  Num examples = 154
  Batch size = 16
***** Running Evaluation *****
  Num examples = 154
  Batch size = 16
***** Running Evaluation *****
  Num examples = 154
  Batch size = 16
***** Running Evaluation *****
  Num examples = 154
  Batch size = 16
***** Running Evaluation *****
  Num examples = 154
  Batch size = 16
***** Running Evaluation *****
  Num examples = 154
  Batch size = 16
***** Running Evaluation *****
  Num examples = 154
  Batch size = 16
***** Running Evaluation *****
  Num examples = 154
  Batch size = 16
***** Running Evaluation *****
  Num examples = 154
  Batch size = 16
***** Running Evalua

TrainOutput(global_step=1450, training_loss=1.265770158438847, metrics={'train_runtime': 213.885, 'train_samples_per_second': 105.43, 'train_steps_per_second': 6.779, 'total_flos': 306403529293824.0, 'train_loss': 1.265770158438847, 'epoch': 50.0})

You can now upload the result of the training to the Hub, just execute this instruction:

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("sgugger/my-awesome-model")
```